In [59]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
import pickle
import os
import numpy as np
from collections import defaultdict
from tqdm import tqdm
from keras.utils.np_utils import to_categorical
import random

In [3]:
i = 0
x = []
y = []

dir = os.path.expanduser('../data/food-101/inception_activations')
for filename in tqdm(sorted(os.listdir(dir))):
    if filename.endswith('.pickle'):
        with open(os.path.join(dir, filename)) as f:
            for path, vector in pickle.load(f).iteritems():
                x.append(vector)
                y.append(i)
        i += 1

x = np.stack(x)
y = np.array(y, dtype=np.int32)

n_classes = np.max(y)+1
input_dim = x.shape[-1]
print input_dim, '->', n_classes


100%|██████████| 70/70 [01:48<00:00,  1.48s/it]


2048 -> 69


In [50]:
# print x.shape, np.expand_dims(y, 1).shape
# y2 = to_categorical(y, 70)
indices = range(len(x))
random.shuffle(indices)
x = x[indices]
y = y[indices]
y2 = to_categorical(y, 70)

In [63]:
model = Sequential()
model.add(BatchNormalization(input_shape=(input_dim,)))
model.add(Dense(512, activation='elu'))

model.add(BatchNormalization(input_shape=(input_dim,)))

model.add(Dropout(0.3))

model.add(Dense(n_classes))
model.add(Activation('softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 2048)              8192      
_________________________________________________________________
dense_38 (Dense)             (None, 512)               1049088   
_________________________________________________________________
batch_normalization_3 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 70)                35910     
_________________________________________________________________
activation_35 (Activation)   (None, 70)                0         
Total params: 1,095,238.0
Trainable params: 1,090,118.0
Non-trainable params: 5,120.0
________________________________________________________

In [65]:
model.fit(x, y2, epochs=10, batch_size=32, validation_split=0.2, verbose=2)


Train on 56000 samples, validate on 14000 samples
Epoch 1/10
13s - loss: 3.3630 - acc: 0.1776 - val_loss: 3.8970 - val_acc: 0.1049
Epoch 2/10
13s - loss: 3.3250 - acc: 0.1879 - val_loss: 3.9081 - val_acc: 0.1082
Epoch 3/10
13s - loss: 3.2866 - acc: 0.1953 - val_loss: 3.9170 - val_acc: 0.1036
Epoch 4/10
13s - loss: 3.2431 - acc: 0.2026 - val_loss: 3.9610 - val_acc: 0.1069
Epoch 5/10
13s - loss: 3.2132 - acc: 0.2094 - val_loss: 3.9904 - val_acc: 0.1039
Epoch 6/10
13s - loss: 3.1726 - acc: 0.2173 - val_loss: 4.0325 - val_acc: 0.1046
Epoch 7/10
13s - loss: 3.1459 - acc: 0.2222 - val_loss: 4.0474 - val_acc: 0.1039
Epoch 8/10
13s - loss: 3.1164 - acc: 0.2279 - val_loss: 4.0958 - val_acc: 0.1016
Epoch 9/10


KeyboardInterrupt: 

In [25]:
print np.argmax(model.predict(x[2000:2010]), axis=1)

[40 40 40 40 40 40 40 40 40 40]
